# Half-twists along the B-L arc

## Modules

In [1]:
import numpy as np
import Permutations as pm
import sympy
from numpy.random import randint as ri
from QuasiCord import *

sympy.init_printing()

import itertools

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

## Integration

In [58]:
def twist(c, hand):
    t, l, b, r = c[0], c[1], c[2], c[3]
    # ---
    delta = (t + l) - (b + r)
    beta = 2 * l + 1 - delta
    z = (b - beta) + (l - beta)

    g = b - l
    G = abs(2 * g - 1)
    # ---
    B = min(beta, 2 * b)
    # ---
    position = None
    bd, ld = 0, 0

    # -- 1 --
    if (b - beta >= 0) and (l - beta >= 0):  # (--> B=beta)
        position = "1"
        if (b - beta == 0) and (l - beta == 0):
            position += "1"
            if hand == "left":
                bd, ld = 1, 0
            else:
                bd, ld = 1, 1
        else:
            position += "2"
            if hand == "left":
                bd, ld = 2 * z + 1, 2 * (b - B)
            else:
                bd, ld = 2 * (l - B) + 1, 2 * z + 1
    # -- 2 --
    elif (b - beta) * (l - beta) <= 0:
        position = "2"
        if g > 0:  # b - beta >= 0, B=beta.  # (--> l-beta < 0)
            position += "1"
            if hand == "left":
                bd, ld = G, G
            else:
                bd, ld = 2 * (l - B) + 1, 2 * (b - B)
        else:  # b-beta < 0 # (--> l-beta <= 0)
            position += "2"
            if hand == "left":
                if 2 * b >= l + 1:
                    position += "1"
                    if 2 * b - l - 1 < -g:
                        position += "1"
                        bd, ld = 2 * (l - B) + 1, -G
                    else:
                        position += "2"
                        bd, ld = -G, -G
                else:
                    position += "2"
                    # ---------------------------------------------------------------------------
                    # >>> NOTE: This is a case where the result of the twist has BR segments. <<<
                    # ---------------------------------------------------------------------------
                    bd, ld = 2 * (l - B) + 1, -B
            else:
                bd, ld = G, G
    # -- 3 --
    elif (b - beta < 0) and (l - beta < 0):
        position = "3"
        bd, ld = 0, 0
    #         if g > 0:
    #             position += "1"
    #             if hand == "left":
    #                 bd, ld = G, G
    #             else:
    #                 if b - B >= 0:
    #                     position += "1"
    #                     bd, ld = -B, 2 * (b - B)
    #                 else:
    #                     position += "2"
    #                     if 2 * b - l >= B:  # <--> (g >= B-b)
    #                         position += "1"
    #                         # ---------------------------------------------------------------------------
    #                         # >>> NOTE: This is a case where the result of the twist has BR segments. <<<
    #                         # ---------------------------------------------------------------------------
    #                         bd, ld = -G, 2 * (b - B)
    #                     else:
    #                         position += "2"
    #                         bd, ld = -G, -G
    #         else:  # g <= 0
    #             position += "2"
    #             if hand == "left":
    #                 position += "1"
    #                 if B < 2 * l + 1 - b:  # <-->  beta < 2l+1-b
    #                     position += "1"
    #                     bd, ld = 2 * (l - beta) + 1, -G
    #                 else:
    #                     position += "2"
    #                     bd, ld = -G, -G
    #             else:
    #                 position += "2"
    #                 bd, ld = G, G
    # -- Exception --
    else:
        print(c, beta, (b - beta, l - beta), hand, "Why??")
    # ---
    try:
        sqr = Square(c)
        # ---
        cn = (t, l + ld, b + bd, r)
        nsqr = Square(cn)
        # ---
        n = cc(sqr)
        nn = cc(nsqr)
        if n != nn:
            print(
                f"{c}:{n} -- {hand}_{position} [g:{g}, beta:{beta}, B:{B}] --> {cn}:{nn}"
            )
    # ---
    except Exception as ex:
        print(f"{c} -- {hand}_{position} [g:{g}, beta:{beta}, B:{B}] --> {cn},{ex}")
        nsqr = sqr
    # ---
    return nsqr, hand + "_" + position

In [59]:
N = 20
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    sqr = square_random_generator(N)
    c = sqr.code
    delta = (c[0] + c[1]) - (c[2] + c[3])
    # ---
    n = cc(sqr)
    sqrnl, pos_l = twist(c, "left")
    sqrnr, pos_r = twist(c, "right")
    nnl = cc(sqrnl)
    nnr = cc(sqrnr)
    # Output exceptions
    if n != nnl:
#         print(f"{c}:{n} -- {pos_l} --> {nnl}")
        error += 1
        pass
    elif n != nnr:
#         print(f"{c}:{n} -- {pos_r} --> {nnr}")
        error += 1
    else:
        pass
    #     if (c[0] - delta == 0) and (c[1]-delta)<0:
    #         print(f"{c} --(ltw)--> {twist(c, 'left').code}")
    #     if (c[1] - delta == 0) and (c[0] > c[1]):
    #         print(f"{c,n} --(ltw, {posil})--> {sqrnl.code,nnl}")
    #         print(f"{c,n} --(rtw, {posir})--> {sqrnr.code,nnr}")
    count += 1
print(count, error)

(17, 4, 1, 15) -- left_222 [g:-3, beta:4, B:2] --> (17, 2, 6, 15),T+L < B+R
(12, 19, 7, 2) -- left_222 [g:-12, beta:17, B:14] --> (12, 5, 18, 2),T+L < B+R
(13, 11, 2, 6) -- left_222 [g:-9, beta:7, B:4] --> (13, 7, 17, 6),T+L < B+R
(10, 10, 3, 4) -- left_222 [g:-7, beta:8, B:6] --> (10, 4, 12, 4),T+L < B+R
(17, 10, 7, 8) -- left_2212 [g:-3, beta:9, B:9] --> (17, 3, 0, 8),T > L+B+R+1
(17, 11, 6, 6) -- left_2211 [g:-5, beta:7, B:7] --> (17, 0, 15, 6),T+L < B+R
(15, 3, 2, 12):2 -- left_2211 [g:-1, beta:3, B:3] --> (15, 0, 3, 12):1
(16, 12, 5, 7):3 -- left_222 [g:-7, beta:9, B:9] --> (16, 3, 12, 7):1
(16, 10, 7, 7) -- left_2212 [g:-3, beta:9, B:9] --> (16, 3, 0, 7),T > L+B+R+1
(16, 10, 6, 7) -- left_2211 [g:-4, beta:8, B:8] --> (16, 1, 11, 7),T+L < B+R
(14, 19, 10, 3):2 -- left_2211 [g:-9, beta:19, B:19] --> (14, 0, 11, 3):1
(19, 7, 4, 13) -- left_2211 [g:-3, beta:6, B:6] --> (19, 0, 7, 13),T+L < B+R
(13, 9, 5, 4) -- left_2211 [g:-4, beta:6, B:6] --> (13, 0, 12, 4),T+L < B+R
(16, 15, 8, 6) 

## Are the left and the right inverses of each other?

In [264]:
N = 20
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tl, pos_tl = twist(c, "left")
    c_tl = sqr_tl.code
    n_tl = cc(sqr_tl)

    sqr_tl_tr, pos_tl_tr = twist(c_tl, "right")
    c_tl_tr = sqr_tl_tr.code
    n_tl_tr = cc(sqr_tl_tr)

    if c != c_tl_tr:  #pos_tl == 'l1':  #
        error += 1
        print(f"{c}:{n} --[{pos_tl}]-->{c_tl}:{n_tl} --[{pos_tl_tr}]-->{c_tl_tr}:{n_tl_tr}")

Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
(6, 14, 9, 1):1 --[left_22]-->(15, 14, 9, 10):1 --[right_3121]-->(6, 14, 9, 0):1
Hi!
Hi!
Hi!
(11, 16, 6, 3):1 --[left_22]-->(26, 16, 6, 18):1 --[right_3121]-->(11, 16, 6, 2):1
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
(11, 14, 4, 3):1 --[left_22]-->(26, 14, 4, 18):1 --[right_3121]-->(11, 14, 4, 2):2
(18, 19, 2, 7):1 --[left_22]-->(39, 19, 2, 28):1 --[right_3121]-->(18, 19, 2, 6):1
(16, 19, 4, 1):1 --[left_22]-->(45, 19, 4, 30):1 --[right_3121]-->(16, 19, 4, 0):1
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!
Hi!


KeyboardInterrupt: 

In [265]:
N = 10
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tr, pos_tr = twist(c, "right")
    c_tr = sqr_tr.code
    n_tr = cc(sqr_tr)

    sqr_tr_tl, pos_tr_tl = twist(c_tr, "left")
    c_tr_tl = sqr_tr_tl.code
    n_tr_tl = cc(sqr_tr_tl)

    if c != c_tr_tl:
        error += 1
        print(f"{c}:{n} --[{pos_tr}]-->{c_tr}:{n_tr} --[{pos_tr_tl}]-->{c_tr_tl}:{n_tr_tl}")

(9, 6, 1, 7):1 --[right_3121]-->(6, 6, 1, 3):1 --[left_22]-->(11, 6, 1, 8):1
(7, 2, 2, 2):5 --[right_311]-->(7, 2, 2, 2):5 --[left_31]-->(16, 2, 2, 11):5
Hi!
(9, 4, 6, 3):2 --[right_311]-->(9, 4, 6, 3):2 --[left_31]-->(20, 4, 6, 14):2
Hi!
(5, 9, 8, 1):1 --[right_22]-->(5, 9, 8, 1):1 --[left_22]-->(12, 9, 8, 8):1
(8, 3, 5, 2):1 --[right_311]-->(8, 3, 5, 2):1 --[left_31]-->(19, 3, 5, 13):1
(3, 7, 6, 2):2 --[right_3121]-->(2, 7, 6, 0):1 --[left_22]-->(5, 7, 6, 3):1
Hi!
(6, 4, 2, 1):2 --[right_22]-->(6, 4, 2, 1):2 --[left_22]-->(15, 4, 2, 10):2
(3, 3, 2, 2):1 --[right_3121]-->(2, 3, 2, 0):2 --[left_22]-->(5, 3, 2, 3):2
Hi!
Hi!
(7, 7, 9, 1):1 --[right_311]-->(7, 7, 9, 1):1 --[left_31]-->(18, 7, 9, 12):1
(6, 3, 3, 2):1 --[right_311]-->(6, 3, 3, 2):1 --[left_31]-->(13, 3, 3, 9):1
(9, 3, 8, 3):1 --[right_311]-->(9, 3, 8, 3):1 --[left_31]-->(20, 3, 8, 14):1
(8, 3, 9, 1):2 --[right_311]-->(8, 3, 9, 1):2 --[left_31]-->(21, 3, 9, 14):2
Hi!
Hi!
(8, 2, 4, 1):1 --[right_311]-->(8, 2, 4, 1):1 --[left_